In [1]:
import importlib
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import datetime
import tensorflow as tf
import gc

from helpers import pre_processing_wlan_utils as preprocess_utils
from helpers import classifier_wlan_spectral_utils as classifier_utils

%matplotlib widget
import matplotlib.pyplot as plt

1 Physical GPUs, 1 Logical GPUs


In [2]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
gc.collect()

Default GPU Device: /device:GPU:0


26

In [3]:
seed = 42
task = "frames"
label = preprocess_utils.label_index[task]
num_classes = preprocess_utils.num_classes[task]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
iq_seqs = [5000]
padding = 'post'
dataset_folder = '../../dataset/waveforms/'
load_model=False
load_dir='../../models/'
load_file = '22042020175144_saved_wavenet_classifier'
gc.collect()

Label id:  0
Num classes in that label:  3


22

In [ ]:
dataset_filename = 'waveforms_16042020_2G_n_unknown_unknown_v2WLAN_CLASS_wlan-frame_balanced.mat'
#X, Y = utils.get_xy(dataset_folder,dataset_filename, num_iq_seq, padding, label, num_classes)
Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)

In [4]:
importlib.reload(preprocess_utils)
importlib.reload(classifier_utils)

#2,6,50
kernel_sizes = [2]
dilation_depths = [3]
n_filters = [32]
now = datetime.datetime.now()
datenow = now.strftime('%d%m%y%H%M%S')
gc.collect()

1 Physical GPUs, 1 Logical GPUs


35

In [5]:
#np.savez('padded_dataset_5000.npz', X_train=X_train, X_val=X_val, X_test=X_test, Y_train=Y_train, Y_val=Y_val, Y_test=Y_test)

In [6]:
npzfile = np.load('padded_dataset_5000.npz')
print("Loading training")
X_train = npzfile['X_train']
Y_train = npzfile['Y_train']
print("Loading validation")
X_val = npzfile['X_val']
Y_val = npzfile['Y_val']
print("Loading testing")
X_test = npzfile['X_test']
Y_test = npzfile['Y_test']
print(X_train.shape,X_val.shape, X_test.shape)
print(Y_train.shape,Y_val.shape, Y_test.shape)

Loading training
Loading validation
Loading testing
(157827, 2, 5000) (33820, 2, 5000) (33821, 2, 5000)
(157827, 3) (33820, 3) (33821, 3)


In [7]:
acc_results=[]
for num_iq_seq in iq_seqs:
    time.sleep(2)
    print("Padding/Truncating sequence")
    X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, num_iq_seq, padding, scale=True)
    time.sleep(2)
    print("Oneshot labels")
    Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)
    print("Performing partitioning of data")
    #X_train, X_val, X_test, Y_train, Y_val, Y_test = classifier_utils.get_xy_4_training(X,Y,seed)
    print(X_train.shape,X_val.shape, X_test.shape)
    print(Y_train.shape,Y_val.shape, Y_test.shape)
    gc.collect()
    for filters in n_filters:
        for ks in kernel_sizes:
            for dd in dilation_depths:
                for t in [1,2,3,4]:
                    print("Creating the model")
                    wnc = classifier_utils.WaveNetClassifier((2,num_iq_seq), (num_classes,), kernel_size = ks, dilation_depth = dd, n_filters = filters, prefix='wlan_classifier')
                    print("Model created")
                    history = wnc.fit(X_train, Y_train, validation_data = (X_val, Y_val),save=True, epochs = 20)
                    test_results = wnc.evaluate(X_test, Y_test)
                    acc_results.append([num_iq_seq, test_results, ks, dd, filters])
                    print(num_iq_seq, test_results)
                    #Save conf matrix
                    file_out_conf_matrix = str(datenow)+"_kernel_"+str(ks)+"_dilatations_"+str(dd)+"_filers_"+str(filters)+"_iq_samples_"+str(num_iq_seq)+"_iter_"+str(t)+"_frames_conf_matrix_test.pdf"
                    Y_pred=np.argmax(wnc.predict(X_test),1)
                    Y_true=np.argmax(Y_test,1)
                    classifier_utils.plot_confusion_matrix_mc(Y_true, Y_pred, ["Mgmt","Ctrl","Data"], file_out_conf_matrix)


Padding/Truncating sequence
Oneshot labels
Performing partitioning of data
(157827, 2, 5000) (33820, 2, 5000) (33821, 2, 5000)
(157827, 3) (33820, 3) (33821, 3)
Creating the model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     [(None, 2, 5000)]    0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 2, 5000, 1)   0           original_input[0][0]             
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 2, 5000, 32)  160         reshaped_input[0][0]             
__________________________________________________________________________________________________
dropout (Drop

ResourceExhaustedError: OOM when allocating tensor with shape[126261,2,5000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:GatherV2]

In [ ]:
test_results = wnc.evaluate(X_test, Y_test)

In [ ]:
print(test_results)

In [ ]:
file_out_conf_matrix = str(datenow)+"_kernel_"+str(kernel_size)+"_dilatations_"+str(dilation_depth)+"_filers_"+str(n_filters)+"_iq_samples_"+str(num_iq_seq)+"_frames_conf_matrix_test.pdf"
print(file_out_conf_matrix)

In [ ]:
Y_pred=np.argmax(wnc.predict(X_test),1)
Y_true=np.argmax(Y_test,1)
classifier_utils.plot_confusion_matrix_mc(Y_true, Y_pred, ["Mgmt","Ctrl","Data"], file_out_conf_matrix)
print(confusion_matrix(Y_true, Y_pred))
print(confusion_matrix(Y_true, Y_pred, normalize='all'))


In [ ]:
res_np = np.array(acc_results)

In [ ]:
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
model = wnc.get_model()
del model